In [235]:
from collections import Counter
import pandas as pd
from IPython.display import display, HTML

# Merging csv files into one datas

In [236]:
devices = pd.read_csv('C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/icij-imddb-2019-07-09/devices-1562662526.csv')
manufacturers = pd.read_csv('C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/icij-imddb-2019-07-09/manufacturers-1562662522.csv')
events =  pd.read_csv('C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/icij-imddb-2019-07-09/events-1562662544.csv')

In [237]:
## merging devices and manufacturers dataframes

#renaming columns in devices dataframe
devices.rename(columns={"id": "device_id"}, inplace=True)
devices.rename(columns={"name": "device_name"}, inplace=True)
devices.rename(columns={"slug": "device_slug"}, inplace=True)
devices.rename(columns={"created_at": "device_created_at"}, inplace=True)
devices.rename(columns={"updated_at": "device_updated_at"}, inplace=True)

#renaming columns in manufacturers dataframe
manufacturers.rename(columns={"id": "manufacturer_id"}, inplace=True)
manufacturers.rename(columns={"name": "manufacturer_name"}, inplace=True)
manufacturers.rename(columns={"slug": "manufacturer_slug"}, inplace=True)
manufacturers.rename(columns={"created_at": "manufacturer_created_at"}, inplace=True)
manufacturers.rename(columns={"updated_at": "manufacturer_updated_at"}, inplace=True)

df = pd.merge(devices,
                 manufacturers,
                 on='manufacturer_id',
                 how='outer')

In [238]:
## merging df and events dataframes

#renaming columns in events dataframe (indicating which original dataset they came from before the merging of datasets)
df.rename(columns={"number": "device_number"}, inplace=True)
df.rename(columns={"country": "device_country"}, inplace=True)
df.rename(columns={"source": "manufacturer_source"}, inplace=True)

#renaming columns in events dataframe
events.rename(columns={"id": "event_id"}, inplace=True)
events.rename(columns={"country": "event_country"}, inplace=True)
events.rename(columns={"source": "event_source"}, inplace=True)
events.rename(columns={"number": "event_number"}, inplace=True)
events.rename(columns={"slug": "event_slug"}, inplace=True)
events.rename(columns={"created_at": "event_created_at"}, inplace=True)
events.rename(columns={"updated_at": "event_updated_at"}, inplace=True)

df_final = pd.merge(df,
                 events,
                 on='device_id',
                 how='outer')

# Dealing with Columns with High Missing Percentage

In [239]:
#Remove columns with missing percentage of 60% or more
df_final = df_final.loc[:, df_final.isnull().mean() < .6]

# Manually Remove Redundant Columns

In [240]:
#Remove columns that have no relations to the outcome column, type (Event Type)
df_final = df_final.drop(['device_id', 'manufacturer_id', 'device_number', 'device_created_at', 'device_updated_at', 'address', 'manufacturer_created_at', 'manufacturer_updated_at', 'event_id', 'authorities_link', 'date_initiated_by_firm', 'uid', 'uid_hash', 'url', 'event_created_at', 'event_updated_at'], axis = 1)

# Exploring Word Frequency of Descriptive Column Texts

In [241]:
#Create copy of the df_final dataset to explore the descriptive columns
df_final_copy = df_final 

#Convert texts in descriptive columns (description, action, data_notes, icij_notes, and reason) to lowercase
df_final_copy['description'] = df_final_copy['description'].str.lower()
df_final_copy['action'] = df_final_copy['action'].str.lower()
df_final_copy['data_notes'] = df_final_copy['data_notes'].str.lower()
df_final_copy['icij_notes'] = df_final_copy['icij_notes'].str.lower()
df_final_copy['reason'] = df_final_copy['reason'].str.lower()

#Remove null from columns
df_final_copy['description'].dropna(inplace=True)
df_final_copy['action'].dropna(inplace=True)
df_final_copy['data_notes'].dropna(inplace=True)
df_final_copy['icij_notes'].dropna(inplace=True)
df_final_copy['reason'].dropna(inplace=True)

In [242]:
#Count the 5 most frequent words from descriptive columns
Counter(" ".join(df_final_copy['description']).split()).most_common(5) #description column

[('the', 51140), ('||', 50935), ('and', 40130), ('for', 28725), ('of', 26126)]

In [243]:
Counter(" ".join(df_final_copy['action']).split()).most_common(5) #action column

[('the', 297063), ('to', 173801), ('and', 109807), ('of', 97222), ('a', 49248)]

In [244]:
Counter(" ".join(df_final_copy['data_notes']).split()).most_common(5) #data_notes column

[('product', 37331),
 ('-', 37013),
 ('code', 35533),
 ('the', 27743),
 ('of', 12856)]

In [245]:
Counter(" ".join(df_final_copy['icij_notes']).split()).most_common(5) #icij_notes column

[('the', 735693),
 ('parent', 328713),
 ('data', 292891),
 ('company', 292887),
 ('from', 273822)]

In [246]:
Counter(" ".join(df_final_copy['reason']).split()).most_common(5) #reason column

[('the', 194711), ('of', 76594), ('to', 59645), ('a', 49291), ('in', 45966)]

In [247]:
#The description, action, and reason columns will be dropped from df_final as they contain largely stop words or symbols as the top five most frequent words
df_final = df_final.drop(['description', 'action', 'reason'], axis = 1)

# Check and Reorganize Values of Columns

In [248]:
#Checking the value counts of certain columns in df_final
df_final['action_classification'].value_counts() #action_classification column

Class 2                    31282
II                          8216
III                         4705
Class 1                     2483
Class II                    2336
Class 3                     2058
Class I                      907
I                            632
Class III                    327
Unclassified Correction        3
Voluntary recall               1
Name: action_classification, dtype: int64

In [249]:
df_final['type'].value_counts() #type column

Recall                   65242
Field Safety Notice      36182
Recall / Safety Alert     4321
Safety alert              3828
Name: type, dtype: int64

In [250]:
#Changing varied-written values of action_classification into uniform values (0 - No classification, 1 - CLass 1, 2 - Class 2, 3 - Class 3)
df_final['action_classification'].fillna('0', inplace = True) 
df_final[['action_classification']] = df_final[['action_classification']].replace(dict.fromkeys(['I', 'Class I', 'Class 1'], '1'))
df_final[['action_classification']] = df_final[['action_classification']].replace(dict.fromkeys(['II','Class II', 'Class 2'], '2'))
df_final[['action_classification']] = df_final[['action_classification']].replace(dict.fromkeys(['III', 'Class III', 'Class 3'], '3'))

#Removing values of Unclassified Correction and Voluntary recall from action_classification column (as there are only a handful, their removal will not affect the final analysis of the data)
df_final = df_final[~(df_final["action_classification"].isin(['Unclassified Correction','Voluntary recall']))]

In [251]:
#Changing values of type column into binary values (1 - recall, 0 - no recall)
df_final[['type']] = df_final[['type']].replace(dict.fromkeys(['Recall', 'Recall / Safety Alert'], '1'))
df_final[['type']] = df_final[['type']].replace(dict.fromkeys(['Field Safety Notice', 'Safety alert'], '0'))

#Remove rows of missing values as type is the outcome variable
df_final = df_final[df_final['type'].notnull()]

# Changing Data Types of Certain Columns

In [252]:
#Changing action_classification and type from object into int
df_final['action_classification'] = df_final['action_classification'].astype(int)
df_final['type'] = df_final['type'].astype(int)

# Changing All Null Values of Object Columns to 'None'

In [253]:
#Replace null values of object columns with string value, 'None'
df_final.fillna(value='None', inplace=True)

# Export Final Dataset into csv

In [254]:
df_final.to_csv (r'C:/Users/melin/Documents/Springboard Data Science Career Track/Capstone Projects/Capstone Project 1/df_final.csv', index = False, header=True)